<a href="https://colab.research.google.com/github/ppg18887/Demo1/blob/master/EffecientDet_Lite_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 64.6 kB of archives.
After this operation, 215 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Fetched 64.6 kB in 1s (81.9 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:

!pip install tflite-runtime
!pip install tflite_model_maker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 23.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 28.7 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3


In [ ]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

Getting efficientdet model

In [ ]:
spec = model_spec.get('efficientdet_lite0')

In [ ]:
#training data with dataset from google cloud(salad data set)
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')

In [ ]:
#Training the model
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
21/21 [==============================] - 73s 1s/step - det_loss: 1.7576 - cls_loss: 1.1345 - box_loss: 0.0125 - reg_l2_loss: 0.0635 - loss: 1.8211 - learning_rate: 0.0090 - gradient_norm: 0.8282 - val_det_loss: 1.6379 - val_cls_loss: 1.0773 - val_box_loss: 0.0112 - val_reg_l2_loss: 0.0635 - val_loss: 1.7014
Epoch 2/50
21/21 [==============================] - 23s 1s/step - det_loss: 1.5940 - cls_loss: 1.0607 - box_loss: 0.0107 - reg_l2_loss: 0.0635 - loss: 1.6575 - learning_rate: 0.0100 - gradient_norm: 1.0935 - val_det_loss: 1.4223 - val_cls_loss: 0.9008 - val_box_loss: 0.0104 - val_reg_l2_loss: 0.0635 - val_loss: 1.4858
Epoch 3/50
21/21 [==============================] - 29s 1s/step - det_loss: 1.3922 - cls_loss: 0.9077 - box_loss: 0.0097 - reg_l2_loss: 0.0635 - loss: 1.4557 - learning_rate: 0.0099 - gradient_norm: 1.8404 - val_det_loss: 1.3421 - val_cls_loss: 0.8482 - val_box_loss: 0.0099 - val_reg_l2_loss: 0.0635 - val_loss: 1.4056
Epoch 4/50
21/21 [======================

In [ ]:
model.evaluate(test_data)

1/1 [==============================] - 8s 8s/step



{'AP': 0.20542076,
 'AP50': 0.34563583,
 'AP75': 0.22947279,
 'APs': -1.0,
 'APm': 0.254003,
 'APl': 0.20526282,
 'ARmax1': 0.16840129,
 'ARmax10': 0.34327206,
 'ARmax100': 0.38093138,
 'ARs': -1.0,
 'ARm': 0.44166666,
 'ARl': 0.38037223,
 'AP_/Baked Goods': 0.03187447,
 'AP_/Salad': 0.5737052,
 'AP_/Cheese': 0.17079881,
 'AP_/Seafood': 0.045285594,
 'AP_/Tomato': 0.2054397}

In [ ]:
model.export(export_dir='.')